In [4]:
import sys

In [5]:
#!{sys.executable} -m pip install psycopg2

In [6]:
#!{sys.executable} -m pip install boto3

In [7]:
import psycopg2

In [8]:
import pandas as pd

In [24]:
import boto3

In [9]:
'''Select email, first_order
            from ecomm_analytics.keen_us_user_profiles
            where first_order >= '10/1/19'
            and email in %s;''' %(items,)

NameError: name 'items' is not defined

In [10]:
conn = psycopg2.connect(user = 'awsuser',
                              password = 'awsPDXhackathon1',
                              host = 'aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com',
                              dbname = 'dev',
                              port = 5439)

cursor = conn.cursor()

cursor.execute(query)
results = pd.DataFrame(cursor.fetchall())
#results.columns = #["Order", "Reason"]#["Email", "Orders", "Revenue"]

cursor.close()
conn.close()

NameError: name 'query' is not defined

In [11]:
import ipywidgets as widgets
print(widgets.Button.on_click.__doc__)

Register a callback to execute when the button is clicked.

        The callback will be called with one argument, the clicked button
        widget instance.

        Parameters
        ----------
        remove: bool (optional)
            Set to true to remove the callback from the list of callbacks.
        


In [12]:
from IPython.display import display

In [13]:

button = widgets.Button(description="Get Data!")
output = widgets.Output()

def on_button_clicked(b):
    output.clear_output()
    with output:
        query= '''Select * from usage_tracker;''' 
        
        conn = psycopg2.connect(user = 'awsuser',
                              password = 'awsPDXhackathon1',
                              host = 'aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com',
                              dbname = 'dev',
                              port = 5439)

        cursor = conn.cursor()

        cursor.execute(query)
        results = pd.DataFrame(cursor.fetchall())
        #results.columns = #["Order", "Reason"]#["Email", "Orders", "Revenue"]

        cursor.close()
        conn.close()
        print (results)

button.on_click(on_button_clicked)

In [14]:
button2 = widgets.Button(description="Press Me")
output2 = widgets.Output()

def on_button2_clicked(b):
    output2.clear_output()
    print ("Button 2")

button2.on_click(on_button2_clicked)

In [15]:
display(button, output)
display(button2, output2)

Button(description='Get Data!', style=ButtonStyle())

Output()

Button(description='Press Me', style=ButtonStyle())

Output()

In [118]:
import psycopg2
import json
import logging
import boto3

logger = logging.getLogger()
logger.setLevel(logging.INFO)

iam1 = boto3.client('iam')


def get_iam_usage():
    
    user_list = get_user_dao()
    #print(user_list)

    for user_name in user_list:   
        #print(user_name[0])
        response_1 = iam1.generate_service_last_accessed_details(
            Arn='arn:aws:iam::132865025212:user/'+user_name[0]
        )
        
        #print(response_1)
        response_dict = json.loads(json.dumps(response_1))
        
        response_2 = iam1.get_service_last_accessed_details(
            JobId=response_dict['JobId']
        )

        response_2_dict = json.loads(json.dumps(response_2, default=str))
        
        if response_2_dict['ServicesLastAccessed']:
            svcs_last_accessed_list = response_2_dict['ServicesLastAccessed']
            #print(svcs_last_accessed_list)
            for svcs_last_accessed in svcs_last_accessed_list:
                if svcs_last_accessed['TotalAuthenticatedEntities'] > 0:
                    update_last_accessed_dao(user_name,svcs_last_accessed['ServiceNamespace'],svcs_last_accessed['LastAuthenticated'])

    return "Successfully updated last accessed dates!!"


def update_last_accessed_dao(user_name, service, last_accessed):
    conn = None
    cur = None
    conn_string = ('dbname=dev user=awsuser password=awsPDXhackathon1 host=aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com port=5439')
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()
    cur.execute('update usage_tracker set last_use_date = %s where user_name = %s and service = %s',(last_accessed,user_name,service,))
    conn.commit()
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()


def get_user_dao():
    conn = None
    cur = None
    conn_string = ('dbname=dev user=awsuser password=awsPDXhackathon1 host=aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com port=5439')
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()
    cur.execute('select distinct user_name from usage_tracker')    
    response = cur.fetchall()
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()
    return response




In [119]:
def get_report():
    conn = psycopg2.connect(user = 'awsuser',
                              password = 'awsPDXhackathon1',
                              host = 'aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com',
                              dbname = 'dev',
                              port = 5439)
    query= '''Select * from usage_tracker order by user_name, service;''' 
    cursor = conn.cursor()
    cursor.execute(query)
    results = pd.DataFrame(cursor.fetchall())
    results.columns = ["User", "Service","Status","Last Accessed"]    
    cursor.close()
    conn.close()    
    return results

def shrinkwrap_users():
    iam2 = boto3.resource('iam')    
    conn = psycopg2.connect(user='awsuser',
                            password='awsPDXhackathon1',
                            host='aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com',
                            dbname='dev',
                            port=5439)    
    get_everything = "SELECT user_name, service FROM usage_tracker WHERE last_use_date < getdate()-30 AND status='Granted';"    
    cursor = conn.cursor()    
    cursor.execute(get_everything)
    results = cursor.fetchall()    
    for expired_permission in results:
        user = iam2.User(expired_permission[0])        
        response = user.detach_policy(PolicyArn=service_policies[expired_permission[1]])
        update_status_query = '''UPDATE usage_tracker SET status='Declined' WHERE user_name='%s' AND service='%s';''' % (expired_permission[0], expired_permission[1])
        cursor.execute(update_status_query)
        conn.commit()    
    cursor.close()
    conn.close()    
    return results

In [120]:
button3 = widgets.Button(description="Create User")
rep_button=widgets.Button(description="Get Report")
sw_button=widgets.Button(description="Shrink Wrap")
lamb_button = widgets.Button(description="Sync Records")
text3 = widgets.Text(value='Enter User Name', description='User', disabled=False)
check1 = widgets.Checkbox(value=False, description='S3 service')
check2 = widgets.Checkbox(value=False, description='Redshift service')
check3=widgets.Checkbox(value=False, description='Lambda service')
output2 = widgets.Output()
rep_output =  widgets.Output()
sw_output = widgets.Output()
lamb_output = widgets.Output()
iam = boto3.resource('iam')
client = boto3.client('iam')

# def on_text_change(change):
#     output2.clear_output()
#     with output2:
#         print(change['new'])

# text3.on_submit(on_text_change)


def on_button3_clicked(b):
    output2.clear_output()
    with output2:
        print(text3.value)
        client.create_user(UserName=text3.value)
        user = iam.User(text3.value)
        
        conn = psycopg2.connect(user='awsuser',
                            password='awsPDXhackathon1',
                            host='aws-pdx-hackathon.c8vcduzakdyw.us-west-2.redshift.amazonaws.com',
                            dbname='dev',
                            port=5439)   
        cursor = conn.cursor()   
        
        if check1.value:
#           user = iam.User(text3.value)
            response = user.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
            insert_new_user = '''insert into usage_tracker values ('%s','%s','%s',getdate());''' % (text3.value, 's3' , 'Granted')    
            cursor.execute(insert_new_user)
            conn.commit()
            print(text3.value+' user has S3 service granted')
        else:
            insert_new_user = '''insert into usage_tracker values ('%s','%s','%s',getdate());''' % (text3.value, 's3' , 'Declined')    
            cursor.execute(insert_new_user)
            conn.commit()
            print(text3.value+' user has S3 service declined')
        if check2.value:
#             user = iam.User(text3.value)
            response = user.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AmazonRedshiftFullAccess')
            insert_new_user = '''insert into usage_tracker values ('%s','%s','%s',getdate());''' % (text3.value, 'redshift' , 'Granted')    
            cursor.execute(insert_new_user)
            conn.commit() 
            print(text3.value+' user has Redshift service granted')
        else:
            insert_new_user = '''insert into usage_tracker values ('%s','%s','%s',getdate());''' % (text3.value, 'redshift' , 'Declined')    
            cursor.execute(insert_new_user)
            conn.commit()
            print(text3.value+' user has Redshift service declined')
        if check3.value:
#             user = iam.User(text3.value)
            response = user.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AWSLambdaFullAccess')
            insert_new_user = '''insert into usage_tracker values ('%s','%s','%s',getdate());''' % (text3.value, 'lambda' , 'Granted')    
            cursor.execute(insert_new_user)
            conn.commit() 
            print(text3.value+' user has Lambda service granted')
        else:
            insert_new_user = '''insert into usage_tracker values ('%s','%s','%s',getdate());''' % (text3.value, 'lambda' , 'Declined')    
            cursor.execute(insert_new_user)
            conn.commit()
            print(text3.value+' user has Lambda service declined')
        
        cursor.close()
        conn.close()

        
def on_rep_button_clicked(b):
    rep_output.clear_output()
    with rep_output:
        print('Getting report...')
        print(get_report())

def on_sw_button_clicked(b):
    sw_output.clear_output()
    with sw_output:
        print('Shrink Wrapping...')
        print(shrinkwrap_users())
        
def on_lamb_button_clicked(b):
    lamb_output.clear_output()
    with lamb_output:
        print('Syncing records on demand...')
        get_iam_usage()
        print('Done!')
        
        

# service_policies = {'redshift': 'arn:aws:iam::aws:policy/AmazonRedshiftFullAccess',
#                     's3': 'arn:aws:iam::aws:policy/AmazonS3FullAccess',
#                     'lambda': 'arn:aws:iam::aws:policy/AWSLambdaFullAccess'}        

def changed(b):
    output2.clear_output()
    with output2:
        print(check1.value)
#         user = iam.User(text3.value)
#         response = user.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
#         print(text3.value+' user has S3 service granted')
def changed1(b):
    output2.clear_output()
    with output2:
        user = iam.User(text3.value)
        response = user.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AmazonRedshiftFullAccess')
        print(text3.value+' user has Redshift service granted')
def changed2(b):
    output2.clear_output()
    with output2:
        user = iam.User(text3.value)
        response = user.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AWSLambdaFullAccess')
        print(text3.value+' user has Lambda service granted')
button3.on_click(on_button3_clicked)
rep_button.on_click(on_rep_button_clicked)
sw_button.on_click(on_sw_button_clicked)
lamb_button.on_click(on_lamb_button_clicked)
# check1.observe(changed)
# check2.observe(changed1)
# check3.observe(changed2)

    



In [121]:
display(button3, text3, check1, check2, check3, output2, rep_button, rep_output, sw_button, sw_output, lamb_button, lamb_output)


Button(description='Create User', style=ButtonStyle())

Text(value='Enter User Name', description='User')

Checkbox(value=False, description='S3 service')

Checkbox(value=False, description='Redshift service')

Checkbox(value=False, description='Lambda service')

...

Output()

Button(description='Shrink Wrap', style=ButtonStyle())

Output()

Button(description='Sync Records', style=ButtonStyle())

Output()